# Project 3, Part 3, Use geodesic distances and zip code and population data to enhance our BART model

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering




In [47]:
import math
import numpy as np
import pandas as pd
import neo4j
import psycopg2

from geographiclib.geodesic import Geodesic

In [48]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [49]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [50]:
cursor = connection.cursor()

In [51]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [52]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, return sum of population within specified miles of station"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
 
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
   
    rows = cursor.fetchall()
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        total_population += population
        
    return(station,int(total_population))
  
    
    

In [22]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station
from stations
order by station

"""

stations= my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

# Data frame of population within 1 mile radius from BART stations
# Train stations located within densely populated areeas can become potential candidates for pick up locations

data=[]

for station in stations["station"]:
    my_station_get_zips(station, 1)
    data.append(my_station_get_zips(station, 1))
    
df = pd.DataFrame(data, columns=["Stations","Populations_within_1_mile"])
df.sort_values("Populations_within_1_mile",ascending = False)


  

,Stations,Populations_within_1_mile
37,Powell Street,140730
22,Glen Park,115068
3,24th Street Mission,108915
6,Balboa Park,106589
35,Pittsburg Center,96081
29,Montgomery Street,85465
15,Downtown Berkeley,82930
43,South Hayward,79235
10,Civic Center,74898
45,Union City,74601


In [53]:
def station_n_miles_radius_berkeley( miles):
    "Find train stations within specified miles of Main Berkeley AGM location"
    
    connection.rollback()
    
    query = """

    select latitude, longitude 
    from stores
    where city = 'Berkeley'

    """
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
     
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
       
    query = "select station from stations "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
   
    rows = cursor.fetchall()
    
    stations =[]
    for row in rows:
        station= row[0]
        stations.append(station)
        
    print(stations)
            

In [54]:
station_n_miles_radius_berkeley(1)

['Ashby', 'Downtown Berkeley', 'Rockridge']


## Ashby is the closest train station to MAIN AGM location. Using geodesic distances in conjuction with population data, and taking into consideration that transfer stations have a lot of foot traffic; the following stations were selected as potential additional pick up locations: Balboa Park, Powell street, Pittsburg Center and Bay Fair.